Installing the dependencies

In [ ]:
!pip install langchain
!pip install transformers
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.6/848.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


# Creation of Job description

In [ ]:
import os
import csv
import random
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the pre-trained model and tokenizer
model_name = "microsoft/phi-2"  # You can use any other compatible model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipeline = pipeline('text-generation', model=model_name, tokenizer=tokenizer, max_new_tokens = 250)

# Load the existing job descriptions from the CSV file
def load_job_descriptions(file_path):
    job_descriptions = []
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            job_descriptions.append(row)
    return job_descriptions

# Save the updated job descriptions to the CSV file
def save_job_descriptions(file_path, job_descriptions):
    fieldnames = ["S.No", "Job Category", "Job Description"]
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(job_descriptions)

# Generate a new job description using the LLM
def generate_job_description(category, existing_description):
    llm = HuggingFacePipeline(pipeline=pipeline)
    prompt = PromptTemplate(
        input_variables=["category", "existing_description"],
        template="Generate a new job description for the {category} category, similar to the following existing description:\n\n{existing_description}Modify the job description by changing the technologies, skills, roles, responsibilities, qualifications etc mentioned, while keeping the overall structure and realistic properties of the job description intact\n\nNew Job Description:"
    )
    new_description = llm(prompt.format(category=category, existing_description=existing_description))
    return new_description.strip()

# Main function to generate job descriptions for each category
def main():
    file_path = "job_descriptions.csv"
    job_descriptions = load_job_descriptions(file_path)
    categories = ["MLE", "SDE", "DE"]
    num_iterations = 12

    for category in categories:
        category_descriptions = [desc for desc in job_descriptions if desc["Job Category"] == category]
        for _ in range(num_iterations):
            random_description = random.choice(category_descriptions)
            new_description = generate_job_description(category, random_description["Job Description"])
            new_id = len(job_descriptions) + 1
            new_job = {"S.No": new_id, "Job Category": category, "Job Description": new_description}
            job_descriptions.append(new_job)

    save_job_descriptions(file_path, job_descriptions)
    print(f"Generated {num_iterations} new job descriptions for each category. Total job descriptions: {len(job_descriptions)}")

if __name__ == "__main__":
    main()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generated 12 new job descriptions for each category. Total job descriptions: 51


Viweing the created job descriptions

In [ ]:
import os
import csv
import random
import anthropic

# Set up the Anthropic client with your API key
api_key = "Enter your API key"
client = anthropic.Anthropic(api_key=api_key)

# Load the existing job descriptions from the CSV file
def load_job_descriptions(file_path):
    job_descriptions = []
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            job_descriptions.append(row)
    return job_descriptions

# Save the updated job descriptions to the CSV file
def save_job_descriptions(file_path, job_descriptions):
    fieldnames = ["S.No", "Job Category", "Job Description"]
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(job_descriptions)

# Generate a new job description using the Claude model
def generate_job_description(category, existing_description):
    prompt = f"Generate a new and different job description for the {category} category, refer this exisitng job description: {existing_description} Please include the following sections in the generated job description:  Job Title: A clear and concise title for the job position.  Roles: The primary roles and duties associated with the position.  Responsibilities: The main responsibilities and tasks the job entails.  Qualifications: The required qualifications, education, and experience for the role.  Skills/Technologies: The necessary skills and technologies the candidate should possess.  Location: The location where the job is based or if it's a remote position. Modify the exisitng job description by changing the technologies, skills, roles, responsibilities, qualifications, and other relevant details while keeping the overall structure and realistic properties of the job description intact. New Job Description : "
    try:
        message = client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=1000,
            temperature=3,
            system="You are a helpful assistant that generates job descriptions.",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        new_description = message.content[0].text
        return new_description
    except Exception as e:
        print(f"Error generating job description: {str(e)}")
        return None

# Main function to generate job descriptions for each category
def main():
    file_path = "job_descriptions.csv"
    job_descriptions = load_job_descriptions(file_path)
    categories = ["MLE", "SDE", "DE"]
    num_iterations = 12  # number of iterations for testing

    for category in categories:
        category_descriptions = [desc for desc in job_descriptions if desc["Job Category"] == category]
        for _ in range(num_iterations):
            random_description = random.choice(category_descriptions)
            new_description = generate_job_description(category, random_description["Job Description"])
            if new_description:
                new_id = len(job_descriptions) + 1
                new_job = {"S.No": new_id, "Job Category": category, "Job Description": new_description}
                job_descriptions.append(new_job)
                print(f"Generated job description for {category}: {new_description}")
            else:
                print(f"Failed to generate job description for {category}")

    save_job_descriptions(file_path, job_descriptions)
    print(f"Generated {num_iterations} new job descriptions for each category. Total job descriptions: {len(job_descriptions)}")

if __name__ == "__main__":
    main()

Generated job description for MLE: Job Title: Computer Vision Engineer

Roles:
- Design and develop cutting-edge computer vision solutions for various applications, including autonomous vehicles, robotics, and augmented reality.
- Collaborate with cross-functional teams to integrate computer vision capabilities into larger systems and products.
- Conduct research and stay up-to-date with the latest advancements in computer vision and deep learning.

Responsibilities:
- Develop and optimize computer vision algorithms and models for tasks such as object detection, tracking, segmentation, and 3D reconstruction.
- Collect, preprocess, and curate large-scale datasets for training and evaluating computer vision models.
- Implement and fine-tune state-of-the-art deep learning architectures using frameworks like PyTorch or TensorFlow.
- Leverage techniques like transfer learning, domain adaptation, and few-shot learning to improve model performance and generalization.
- Integrate computer visi

In [ ]:
import pandas as pd

data = pd.read_csv('resumes.csv')
print(data.head())

# Generation of Relevant and Irrelevant Resume Given a Job Description

In [ ]:
import os
import csv
import random
import anthropic
import re

# Set up the Anthropic client with your API key
api_key = "Enter your API key here"
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions from the CSV file
def load_job_descriptions(file_path):
    job_descriptions = []
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            job_descriptions.append(row)
    return job_descriptions

# Save the generated resumes to the CSV file
def save_resumes(file_path, resumes):
    fieldnames = ["S.No", "Job Category", "Job Description", "Resume", "Qualifications", "Previous Work Experience",
                  "Years of Experience", "Domain Expertise", "Position Name", "Projects Done",
                  "Skills", "Certifications", "Relevance"]
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(resumes)

# Generate a resume based on the job description
def generate_resume(job_description, relevance):
    if relevance:
        prompt = f"Generate a highly relevant resume for the following job description:\n\n{job_description}\n\nEnsure that the following KEYS are present:  1. years of experience  2.qualifications, 3. previous work experience, 4. domain expertise, 5. position name, 6. Projects Done (This exact key words only), 7. skills, 8. certifications .\n- The resume highlights key achievements and projects that align with the job requirements. \n\nResume:"
    else:
        prompt = f"Generate a somewhat irrelevant resume in the domain of IT, Could be any technology apart from what is mentioned in the job dsecription. \n\n{job_description} NOTE : \n\nEnsure that :\n- The resume differs or lacks some of the essentials job requirements, INCLUDE ALL THESE KEYS  like 1. qualifications, 2. previous work experience, 3. years of experience, 4. domain expertise, 5. position name, 6. Projects Done (This exact key words only), 7. skills 8.certifications. \n\nResume:"

    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that generates resumes. And your generated response conatains only what is asked in the prompt and does need to include things like here is your highly relevant/irrelevant resume, Just the response is enough",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    resume = message.content[0].text
    return resume


# Extract features based on the resumes generated
def extract_features(resume):
    prompt = f"Given the following resume, extract the following relevant features and return them as a Python dictionary literal, Keep in mind the generated reponse from you should strictly only contain Python dictionary literal :\n\nResume:\n{resume} \n\nExtract the following features from the resume: \n- Skills \n- Qualifications\n- Previous Work Experience\n- Years of Experience (if related to the job) \n- Domain Expertise (if mentioned)\n- Position Name\n- Projects Done mentioned \n- Certifications \n\n Only If any of the features are not present in the resume, leave them as empty strings. Double check if all the features are extracted properly. \n Return the extracted features as a Python dictionary literal."
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that extracts features from resumes and the generated response contains and returns them strictly as Python dictionary literal only.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    features_dict_literal = message.content[0].text.strip()
    features_dict = eval(features_dict_literal)
    return features_dict



# Main function to generate resumes for each job description
def main():
    input_file = "job_descriptions.csv"
    output_file = "resumes.csv"
    job_descriptions = load_job_descriptions(input_file)
    resumes = []
    num_relevant_resumes = 1  # Number of relevant resumes to generate for each job description
    num_irrelevant_resumes = 1  # Number of irrelevant resumes to generate for each job description

    # Generate resumes for only the first job description
    job = job_descriptions[0]
    s_no = job["S.No"]
    category = job["Job Category"]
    description = job["Job Description"]

    for _ in range(num_relevant_resumes):
      relevant_resume = generate_resume(description, True)
      relevant_features = extract_features(relevant_resume)
      resumes.append({"S.No": s_no, "Job Category": category, "Job Description": description,
                      "Resume": relevant_resume, "Qualifications": relevant_features.get("Qualifications", ""),
                      "Previous Work Experience": relevant_features.get("Previous Work Experience", ""),
                      "Years of Experience": relevant_features.get("Years of Experience", ""),
                      "Domain Expertise": relevant_features.get("Domain Expertise", ""),
                      "Position Name": relevant_features.get("Position Name", ""),
                      "Projects Done": relevant_features.get("Projects Done", ""),
                      "Skills": relevant_features.get("Skills", ""),
                      "Certifications": relevant_features.get("Certifications", ""),
                      "Relevance": "Yes"})

    for _ in range(num_irrelevant_resumes):
      irrelevant_resume = generate_resume(description, False)
      irrelevant_features = extract_features(irrelevant_resume)
      resumes.append({"S.No": s_no, "Job Category": category, "Job Description": description,
                      "Resume": irrelevant_resume, "Qualifications": irrelevant_features.get("Qualifications", ""),
                      "Previous Work Experience": irrelevant_features.get("Previous Work Experience", ""),
                      "Years of Experience": irrelevant_features.get("Years of Experience", ""),
                      "Domain Expertise": irrelevant_features.get("Domain Expertise", ""),
                      "Position Name": irrelevant_features.get("Position Name", ""),
                      "Projects Done": irrelevant_features.get("Projects Done", ""),
                      "Skills": irrelevant_features.get("Skills", ""),
                      "Certifications": irrelevant_features.get("Certifications", ""),
                      "Relevance": "No"})


    save_resumes(output_file, resumes)
    print(f"Generated {len(resumes)} resumes for the first job description.")

if __name__ == "__main__":
    main()

Text Only : {
    "Skills": ["Python", "C#", "JavaScript", "TypeScript", "NumPy", "Pandas", "Scikit-learn", "TensorFlow", "PyTorch", "React", "GraphQL", "Azure", "AWS", "Google Cloud", "Docker", "Kubernetes", "Terraform", "Ansible", "Jenkins", "LLMs (Azure, Google, etc.)", "Vertext AI", "Azure AI", "IBM WatsonX", "Hugging Face"],
    "Qualifications": ["Bachelor's degree in Computer Science, expected graduation in Spring 2025", "Strong academic background in Machine Learning, Artificial Intelligence, and Data Science"],
    "Previous Work Experience": ["Machine Learning Intern, XYZ Corp. (Summer 2023) - Assisted in the development and deployment of machine learning models for predictive maintenance in manufacturing. - Utilized cloud platforms like Azure and AWS for model training and deployment."],
    "Years of Experience": "Incoming Graduate",
    "Domain Expertise": "- Strong foundation in machine learning algorithms, including supervised and unsupervised learning techniques. - Prof

In [ ]:
import os
import csv
import random
import anthropic
import re

# Set up the Anthropic client with your API key
api_key = "Enter your API key here"
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions from the CSV file
def load_job_descriptions(file_path):
    job_descriptions = []
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            job_descriptions.append(row)
    return job_descriptions

# Save the generated resumes to the CSV file
def save_resumes(file_path, resumes):
    fieldnames = ["S.No", "Job Category", "Job Description", "Resume", "Qualifications", "Previous Work Experience",
                  "Years of Experience", "Domain Expertise", "Position Name", "Projects Done",
                  "Skills", "Certifications", "Relevance"]
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(resumes)

# Generate a resume based on the job description
def generate_resume(job_description, relevance):
    if relevance:
        prompt = f"Generate a highly relevant resume for the following job description:\n\n{job_description}\n\nEnsure that the following KEYS are present: 1. years of experience 2.qualifications, 3. previous work experience, 4. domain expertise, 5. position name, 6. Projects Done (This exact key words only), 7. skills, 8. certifications .\n- The resume highlights key achievements and projects that align with the job requirements. \n\nResume:"
    else:
        prompt = f"Generate a somewhat irrelevant resume in the domain of IT, Could be any technology apart from what is mentioned in the job dsecription. \n\n{job_description} NOTE : \n\nEnsure that :\n- The resume differs or lacks some of the essentials job requirements, INCLUDE ALL THESE KEYS like 1. qualifications, 2. previous work experience, 3. years of experience, 4. domain expertise, 5. position name, 6. Projects Done (This exact key words only), 7. skills 8.certifications. \n\nResume:"

    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that generates resumes. And your generated response conatains only what is asked in the prompt and does need to include things like here is your highly relevant/irrelevant resume, Just the response is enough",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    resume = message.content[0].text
    return resume

# Extract features based on the resumes generated
def extract_features(resume):
    prompt = f"Given the following resume, extract the following relevant features and return them as a Python dictionary literal, Keep in mind the generated reponse from you should strictly only contain Python dictionary literal :\n\nResume:\n{resume} \n\nExtract the following features from the resume: \n- Skills \n- Qualifications\n- Previous Work Experience\n- Years of Experience (if related to the job) \n- Domain Expertise (if mentioned)\n- Position Name\n- Projects Done mentioned \n- Certifications \n\n Only If any of the features are not present in the resume, leave them as empty strings. Double check if all the features are extracted properly. \n Return the extracted features as a Python dictionary literal."
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that extracts features from resumes and the generated response contains and returns them strictly as Python dictionary literal only.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    features_dict_literal = message.content[0].text.strip()
    features_dict = eval(features_dict_literal)
    return features_dict

# Main function to generate resumes for each job description
def main():
    input_file = "job_descriptions.csv"
    output_file = "resumes.csv"
    job_descriptions = load_job_descriptions(input_file)
    resumes = []
    num_relevant_resumes = 1  # Number of relevant resumes to generate for each job description
    num_irrelevant_resumes = 1  # Number of irrelevant resumes to generate for each job description

    for job in job_descriptions:
        s_no = job["S.No"]
        category = job["Job Category"]
        description = job["Job Description"]

        for _ in range(num_relevant_resumes):
            relevant_resume = generate_resume(description, True)
            relevant_features = extract_features(relevant_resume)
            resumes.append({"S.No": s_no, "Job Category": category, "Job Description": description,
                            "Resume": relevant_resume, "Qualifications": relevant_features.get("Qualifications", ""),
                            "Previous Work Experience": relevant_features.get("Previous Work Experience", ""),
                            "Years of Experience": relevant_features.get("Years of Experience", ""),
                            "Domain Expertise": relevant_features.get("Domain Expertise", ""),
                            "Position Name": relevant_features.get("Position Name", ""),
                            "Projects Done": relevant_features.get("Projects Done", ""),
                            "Skills": relevant_features.get("Skills", ""),
                            "Certifications": relevant_features.get("Certifications", ""),
                            "Relevance": "Yes"})

        for _ in range(num_irrelevant_resumes):
            irrelevant_resume = generate_resume(description, False)
            irrelevant_features = extract_features(irrelevant_resume)
            resumes.append({"S.No": s_no, "Job Category": category, "Job Description": description,
                            "Resume": irrelevant_resume, "Qualifications": irrelevant_features.get("Qualifications", ""),
                            "Previous Work Experience": irrelevant_features.get("Previous Work Experience", ""),
                            "Years of Experience": irrelevant_features.get("Years of Experience", ""),
                            "Domain Expertise": irrelevant_features.get("Domain Expertise", ""),
                            "Position Name": irrelevant_features.get("Position Name", ""),
                            "Projects Done": irrelevant_features.get("Projects Done", ""),
                            "Skills": irrelevant_features.get("Skills", ""),
                            "Certifications": irrelevant_features.get("Certifications", ""),
                            "Relevance": "No"})

    save_resumes(output_file, resumes)
    print(f"Generated {len(resumes)} resumes for all job descriptions.")

if __name__ == "__main__":
    main()

Generated 102 resumes for all job descriptions.


# END OF JOB DESCRIPTION, RESUME, RELEVANT CATEGORIES GENERATION

In [ ]:
import pandas as pd

data = pd.read_csv('resumes.csv')

In [ ]:
data.columns.values.tolist()

['S.No',
 'Job Category',
 'Job Description',
 'Resume',
 'Qualifications',
 'Previous Work Experience',
 'Years of Experience',
 'Domain Expertise',
 'Position Name',
 'Projects Done',
 'Skills',
 'Certifications',
 'Relevance']

In [ ]:
new_data = data.drop(['Qualifications',
 'Previous Work Experience',
 'Years of Experience',
 'Domain Expertise',
 'Position Name',
 'Projects Done',
 'Skills',
 'Certifications',
 'Relevance'], axis= 1)

In [ ]:
print(new_data)

     S.No Job Category                                    Job Description  \
0       1          MLE  What you will be doing:  Develop, deploy and o...   
1       1          MLE  What you will be doing:  Develop, deploy and o...   
2       2          MLE  Responsibilities Design, develop, and deploy m...   
3       2          MLE  Responsibilities Design, develop, and deploy m...   
4       3          MLE   ABOUT THE ROLE The AI/ML Engineer will be res...   
..    ...          ...                                                ...   
97     49           DE  Job Title: Data Engineer - Artificial Intellig...   
98     50           DE  Job Title: Data Engineering Architect\n\nRoles...   
99     50           DE  Job Title: Data Engineering Architect\n\nRoles...   
100    51           DE  Job Title: Data Engineering Lead\n\nRoles:\n- ...   
101    51           DE  Job Title: Data Engineering Lead\n\nRoles:\n- ...   

                                                Resume  
0    Position Name

In [ ]:
new_data.to_csv('file1.csv')

# Using a Weighted Score approach for evaluating the resumes given the job descriptions


In [ ]:
import os
import csv
import anthropic

# Set up the Anthropic client with your API key
api_key = "Enter your API key here"
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions and resumes from the CSV file
def load_data(file_path):
    data = []
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)
    return data

# Save the scores and explanations to the CSV file
def save_scores(file_path, data):
    fieldnames = data[0].keys()
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Generate score and explanation for a resume based on the job description
def generate_score_and_explanation(job_description, resume):
    prompt = f"""
    Given the following job description and resume, evaluate the relevance of the resume to the job requirements and provide a score between 0 and 1, along with a brief explanation justifying the score.

    Job Description:
    {job_description}

    Resume:
    {resume}

    Guidelines for scoring:
    - Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.
    - This idea is like conditional probability where, given the job description, to find the score of the resume.
    - Assess the relevance of the candidate's background to the specific needs mentioned in the job description.
    - Use the following weighted schema to prioritize different aspects of the resume:
      - Skills: 25%
      - Work Experience: 35%
      - Qualifications: 5%
      - Projects: 30%
      - Additional Relevant Factors: 5%
    - Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.
    - Generate a brief explanation highlighting the key factors that influenced the score, including strengths and any gaps or areas for improvement.

    Please provide the score and explanation in STRICTLY IN THE FOLLOWING format ONLY:
    {{
        "score": <score>,
        "explanation": "<explanation>"
    }}
    """

    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that evaluates resumes based on job descriptions.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    result = response.content[0].text.strip()
    return eval(result)

# Main function to process the data and generate scores and explanations
def main():
    input_file = "data.csv"
    output_file = "scores.csv"

    data = load_data(input_file)

    for row in data:
        job_description = row["Job Description"]
        resume = row["Resume"]

        result = generate_score_and_explanation(job_description, resume)
        row["Score"] = result["score"]
        row["Explanation"] = result["explanation"]

    save_scores(output_file, data)
    print(f"Scores and explanations generated and saved to {output_file}.")

if __name__ == "__main__":
    main()

Scores and explanations generated and saved to scores.csv.


In [ ]:
'''

CHAIN OF THOUGHT PROMPTING ATTEMPT
import os
import csv
import anthropic

# Set up the Anthropic client with your API key
api_key = "Your API key here"
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions and resumes from the CSV file
def load_data(file_path):
    data = []
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)
    return data

# Save the scores and explanations to the CSV file
def save_scores(file_path, data):
    fieldnames = data[0].keys()
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Generate score and explanation for a resume based on the job description
def generate_score_and_explanation(job_description, resume):
    prompt = f"""
    Given the following job description and resume, evaluate the relevance of the resume to the job requirements and provide a score between 0 and 1, along with a detailed explanation justifying the score.

    Job Description:
    {job_description}

    Resume:
    {resume}

    Guidelines for scoring:
    - Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.
    - This idea is like conditional probability where, given the job description, to find the score of the resume.
    - Assess the relevance of the candidate's background to the specific needs mentioned in the job description.
    - Use the following weighted schema to prioritize different aspects of the resume:
      - Skills: 25%
      - Work Experience: 35%
      - Qualifications: 5%
      - Projects: 30%
      - Additional Relevant Factors: 5%
    - Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.

    Please provide a step-by-step explanation of your thought process in evaluating the resume. Break down the evaluation into smaller steps and provide reasoning for each step. Consider the following points in your explanation:
    - How well do the candidate's skills align with the job requirements?
    - Is the candidate's work experience relevant to the job description?
    - Do the candidate's qualifications meet the job requirements?
    - Are the candidate's projects relevant to the job description?
    - Are there any additional relevant factors that influence the score?
    Everything can be put in one paragraph.

    After providing the step-by-step explanation, please provide the final score and a brief summary of the key factors that influenced the score, including strengths and any gaps or areas for improvement.

    Please provide the score and explanation in STRICTLY IN THE FOLLOWING format ONLY:
    {{
        "score": <score>,
        "explanation": "<step-by-step explanation>\\n\\n<summary>"
    }}
    """

    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that evaluates resumes based on job descriptions.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    result = response.content[0].text.strip()
    return eval(result)

# Main function to process the data and generate scores and explanations
def main():
    input_file = "data.csv"
    output_file = "scores.csv"

    data = load_data(input_file)

    for row in data:
        job_description = row["Job Description"]
        resume = row["Resume"]

        result = generate_score_and_explanation(job_description, resume)
        row["Score"] = result["score"]
        row["Explanation"] = result["explanation"]

    save_scores(output_file, data)
    print(f"Scores and explanations generated and saved to {output_file}.")

if __name__ == "__main__":
    main()
'''

In [ ]:
import csv
import json

def convert_dataset(input_file, output_file):
    with open(input_file, "r") as file:
        reader = csv.DictReader(file)
        data = list(reader)

    converted_data = []
    for row in data:
        job_description = row["Job Description"]
        resume = row["Resume"]
        score = row["Score"]
        explanation = row["Explanation"]

        instruction = f"Job Description:\n{job_description}\n\nResume:\n{resume}\n\nGuidelines for scoring:\n- Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.\n- This idea is like conditional probability where, given the job description, to find the score of the resume.\n- Assess the relevance of the candidate's background to the specific needs mentioned in the job description.\n- Use the following weighted schema to prioritize different aspects of the resume:\n  - Skills: 25%\n  - Work Experience: 35%\n  - Qualifications: 5%\n  - Projects: 30%\n  - Additional Relevant Factors: 5%\n- Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.\n- Generate a brief explanation highlighting the key factors that influenced the score, including strengths and any gaps or areas for improvement.\n\nPlease provide the score, explanation, and a breakdown of how the weighted schema was applied to arrive at the score. Use the following format:\n{{\n    \"score\": <score>,\n    \"explanation\": \"<explanation>\",\n    \"weighted_schema_breakdown\": \"<breakdown>\"\n}}"

        response = f"{{\n    \"score\": {score},\n    \"explanation\": \"{explanation}\",\n    \"weighted_schema_breakdown\": \"The score was calculated using the following weighted schema:\\n- Skills: 25% (Candidate's skills aligned with the job requirements)\\n- Work Experience: 35% (Candidate's work experience is relevant to the role)\\n- Qualifications: 5% (Candidate meets the required qualifications)\\n- Projects: 30% (Candidate has completed projects similar to the job responsibilities)\\n- Additional Relevant Factors: 5% (Candidate possesses additional relevant factors)\\nThe weighted scores from each category were summed up to arrive at the final score.\"\n}}"

        converted_row = {
            "inputs": f"<s>### Instruction:\n{instruction}\n\n### Response:\n{response}</s>"
            text = f"### Question: {example['input']}\n ### Answer: {example['output']}"

        }
        converted_data.append(converted_row)

    with open(output_file, "w") as file:
        json.dump(converted_data, file, indent=2)

    print(f"Dataset converted and saved to {output_file}.")
    print(f"Total number of rows in the converted dataset: {len(converted_data)}")

# Usage
input_file = "data.csv"
output_file = "converted_dataset.json"
convert_dataset(input_file, output_file)

Dataset converted and saved to converted_dataset.json.
Total number of rows in the converted dataset: 102


In [ ]:
import os
import csv
import anthropic
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set up the Anthropic client with your API key
api_key = os.environ.get("ANTHROPIC_API_KEY")  # Store the API key as an environment variable
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions and resumes from the CSV file
def load_data(file_path):
    data = []
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if "Job Description" in row and "Resume" in row:
                data.append(row)
            else:
                logging.warning(f"Skipping row due to missing 'Job Description' or 'Resume' column: {row}")
    return data

# Save the scores and explanations to the CSV file
def save_scores(file_path, data):
    fieldnames = data[0].keys()
    with open(file_path, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Generate score and explanation for a resume based on the job description
def generate_score_and_explanation(job_description, resume):
    prompt = f"""
    Given the following job description and resume, evaluate the relevance of the resume to the job requirements and provide a score between 0 and 1, along with a brief explanation justifying the score.

    Job Description:
    {job_description}

    Resume:
    {resume}

    Guidelines for scoring:
    - Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.
    - This idea is like conditional probability where, given the job description, to find the score of the resume.
    - Assess the relevance of the candidate's background to the specific needs mentioned in the job description.
    - Use the following weighted schema to prioritize different aspects of the resume:
      - Skills: 25%
      - Work Experience: 35%
      - Qualifications: 5%
      - Projects: 30%
      - Additional Relevant Factors: 5%
    - Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.
    - Generate a brief explanation highlighting the key factors that influenced the score, including strengths and any gaps or areas for improvement.
    - Break down the explanation based on the weighted schema factors, providing insights into how each factor contributes to the overall score.

    Please provide the score and explanation in STRICTLY IN THE FOLLOWING format ONLY:
    {{
    "score": <score>,
    "explanation": "<explanation>",
    "skills_score": <skills_score>,
    "work_experience_score": <work_experience_score>,
    "qualifications_score": <qualifications_score>,
    "projects_score": <projects_score>,
    "additional_factors_score": <additional_factors_score>
    }}
    """
    try:
        response = client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=4000,
            temperature=1,
            system="You are a helpful assistant that evaluates resumes based on job descriptions.",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        result = response.content[0].text.strip()
        return eval(result)
    except Exception as e:
        logging.error(f"Error generating score and explanation: {str(e)}")
        return None

# Main function to process the data and generate scores and explanations
def main():
    input_file = "data.csv"
    output_file = "scores.csv"
    data = load_data(input_file)
    for row in data:
        job_description = row["Job Description"]
        resume = row["Resume"]
        result = generate_score_and_explanation(job_description, resume)
        if result:
            row["Score"] = result["score"]
            row["Explanation"] = result["explanation"]
            row["Skills Score"] = result["skills_score"]
            row["Work Experience Score"] = result["work_experience_score"]
            row["Qualifications Score"] = result["qualifications_score"]
            row["Projects Score"] = result["projects_score"]
            row["Additional Factors Score"] = result["additional_factors_score"]
        else:
            logging.warning(f"Skipping row due to error in generating score and explanation: {row}")
    save_scores(output_file, data)
    logging.info(f"Scores and explanations generated and saved to {output_file}.")

if __name__ == "__main__":
    main()

# SCORE AND EXPLANATION CREATION (WITH IMRPOVEMENTS BY PROMPT TUNING)

In [ ]:
import os
import csv
import anthropic
import openpyxl

# Set up the Anthropic client with your API key
api_key = "Enter your API key here"
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions and resumes from the XLSX file
def load_data(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    data = []
    headers = [cell.value for cell in sheet[1]]
    for row in sheet.iter_rows(min_row=2, values_only=True):
        data.append(dict(zip(headers, row)))
    return data

# Save the scores and explanations to the XLSX file
def save_scores(file_path, data):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    headers = list(data[0].keys())
    sheet.append(headers)
    for row in data:
        sheet.append(list(row.values()))
    workbook.save(file_path)

# Generate score and explanation for a resume based on the job description
def generate_score_and_explanation(job_description, resume, qualification, work_experience, years_of_experience, domain_expertise, position_name, projects_done, skills, certifications):
    prompt = f"""
    Given the following job description and resume details, evaluate the relevance of the resume to the job requirements and provide a score between 0 and 1, along with a detailed explanation justifying the score.

    Job Description:
    {job_description}

    Resume Details:
    - Qualification: {qualification}
    - Previous Work Experience: {work_experience}
    - Years of Experience: {years_of_experience}
    - Domain Expertise: {domain_expertise}
    - Position Name: {position_name}
    - Projects Done: {projects_done}
    - Skills: {skills}
    - Certifications: {certifications}

    Actual Resume for Reference : {resume}

    Guidelines for scoring:

    - Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.
    - This idea is like conditional probability where, given the job description, find the score of the resume.
    - Assess the relevance of the candidate's background to the specific needs mentioned in the job description.
    - Use the following weighted schema to prioritize different aspects of the resume:
      - Previous Work Experience: 30%
      - Projects Done: 25%
      - Skills: 20%
      - Certifications: 10%
      - Qualifications: 5%
      - Years of Experience: 5%
      - Domain Expertise: 3%
      - Position Name: 2%
    - Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.

    Guidelines for explanation:
    - Generate a detailed explanation highlighting the key factors that influenced the score rhough the weighted schema apporach mentioned.

    Please provide the score and explanation STRICTLY IN THE FOLLOWING FORMAT ONLY:
    {{
    "score": <score>,
    "explanation": "<explanation>"
    }}
    """
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that evaluates resumes based on job descriptions.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    result = response.content[0].text.strip()
    return eval(result)

# Main function to process the data and generate scores and explan

In [ ]:
import os
import pandas as pd
import anthropic
import re

# Set up the Anthropic client with your API key
api_key = "Enter your API key here"
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions and resumes from the xlsx file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Save the scores and explanations to the xlsx file
def save_scores(file_path, data):
    data.to_excel(file_path, index=False)

# Extract score and explanation from the response using regular expressions
def extract_score_and_explanation(response_text):
    score_match = re.search(r'"score":\s*(\d+\.\d+)', response_text)
    explanation_match = re.search(r'"explanation":\s*"(.+?)"', response_text, re.DOTALL)

    if score_match and explanation_match:
        score = float(score_match.group(1))
        explanation = explanation_match.group(1).strip()
        return {"score": score, "explanation": explanation}
    else:
        return {"score": 0, "explanation": "Failed to extract score and explanation from the response"}

# Generate score and explanation for a resume based on the job description and additional features
def generate_score_and_explanation(job_description, qualification, work_experience, years_of_experience, domain_expertise, position_name, projects_done, skills, certifications):
    prompt = f"""
    Given the following job description, and additional features, evaluate the relevance of the resume to the job requirements and provide a score between 0 and 1, along with a brief explanation justifying the score.
    Job Description:
    {job_description}
    Extracted Features to be considered for Weighted Schema:
    - Qualification: {qualification}
    - Previous Work Experience: {work_experience}
    - Years of Experience: {years_of_experience}
    - Domain Expertise: {domain_expertise}
    - Position Name: {position_name}
    - Projects Done: {projects_done}
    - Skills: {skills}
    - Certifications: {certifications}
    VERY IMPORTANT NOTE : Please provide the score and explanation STRICTLY STRICTLY IN THE FOLLOWING format ONLY :
    {{
    "score": <score>,
    "explanation": "<explanation content>"
    }}
    Guidelines for explanation:
    - Generate a detailed explanation highlighting the key factors on how it individually contributed to the overall score by the weighted schema approach.
    - The score you give for each feature of the resume is relative and compared to the given job description and should also be included in the explanation. For Eg: (10% match only in the previous work experience, 0% for certification, and say a how individual features contributed in the total score)
    - Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.
    Guidelines for scoring:
    - Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.
    - This idea is like conditional probability where, given the job description, to find the score of the resume.
    - Assess the relevance of the candidate's background to the specific needs mentioned in the job description.
    - Use the following weighted schema to prioritize different aspects of the resume:
      - Previous Work Experience: 30%
      - Projects Done: 25%
      - Skills: 20%
      - Certifications: 10%
      - Qualifications: 5%
      - Years of Experience: 5%
      - Domain Expertise: 3%
      - Position Name: 2%
    """
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that evaluates resumes based on job descriptions.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    response_text = response.content[0].text.strip()
    result = extract_score_and_explanation(response_text)
    return result

# Main function to process the data and generate scores and explanations
def main():
    input_file = "data.xlsx"
    output_file = "scores.xlsx"
    data = load_data(input_file)
    scores = []
    explanations = []
    for _, row in data.iterrows():
        job_description = row["Job Description"]
        resume = row["Resume"]
        qualification = row["Qualifications"]
        work_experience = row["Previous Work Experience"]
        years_of_experience = row["Years of Experience"]
        domain_expertise = row["Domain Expertise"]
        position_name = row["Position Name"]
        projects_done = row["Projects Done"]
        skills = row["Skills"]
        certifications = row["Certifications"]
        result = generate_score_and_explanation(job_description, qualification, work_experience, years_of_experience, domain_expertise, position_name, projects_done, skills, certifications)
        scores.append(result["score"])
        explanations.append(result["explanation"])
    data["Score"] = scores
    data["Explanation"] = explanations
    save_scores(output_file, data)
    print(f"Scores and explanations generated and saved to {output_file}.")

if __name__ == "__main__":
    main()

Scores and explanations generated and saved to scores.xlsx.


In [ ]:
import os
import pandas as pd
import anthropic
import re

# Set up the Anthropic client with your API key
api_key = "Enter your API key here"
client = anthropic.Anthropic(api_key=api_key)

# Load the job descriptions and resumes from the xlsx file
def load_data(file_path):
    data = pd.read_excel(file_path)
    return data

# Save the scores and explanations to the xlsx file
def save_scores(file_path, data):
    data.to_excel(file_path, index=False)

# Extract score and explanation from the response using regular expressions
def extract_score_and_explanation(response_text):
    score_match = re.search(r'"score":\s*(\d+\.\d+)', response_text)
    explanation_match = re.search(r'"explanation":\s*"(.+?)"', response_text, re.DOTALL)
    if score_match and explanation_match:
        score = float(score_match.group(1))
        explanation = explanation_match.group(1).strip()
        return {"score": score, "explanation": explanation}
    else:
        return {"score": 0, "explanation": "Failed to extract score and explanation from the response"}

# Generate score and explanation for a resume based on the job description and additional features
def generate_score_and_explanation(job_description, qualification, work_experience, years_of_experience, domain_expertise, position_name, projects_done, skills, certifications):
    prompt = f"""
    Given the following job description, and additional features, evaluate the relevance of the resume to the job requirements and provide a score between 0 and 1, along with a brief explanation justifying the score.
    Job Description:
    {job_description}
    Extracted Features to be considered for Weighted Schema:
    - Qualification: {qualification}
    - Previous Work Experience: {work_experience}
    - Years of Experience: {years_of_experience}
    - Domain Expertise: {domain_expertise}
    - Position Name: {position_name}
    - Projects Done: {projects_done}
    - Skills: {skills}
    - Certifications: {certifications}
    VERY IMPORTANT NOTE : Please provide the score and explanation STRICTLY STRICTLY IN THE FOLLOWING format ONLY :
    {{
    "score": <score>,
    "explanation": "<explanation content>"
    }}
    Guidelines for explanation:
    - Generate a detailed explanation highlighting the key factors on how it individually contributed to the overall score by the weighted schema approach.
    - The score you give for each feature of the resume is relative and compared to the given job description and should also be included in the explanation. For Eg: (10% match only in the previous work experience, 0% for certification, and say a how individual features contributed in the total score)
    - Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.
    Guidelines for scoring:
    - Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.
    - This idea is like conditional probability where, given the job description, to find the score of the resume.
    - Assess the relevance of the candidate's background to the specific needs mentioned in the job description.
    - Use the following weighted schema to prioritize different aspects of the resume:
    - Previous Work Experience: 30%
    - Projects Done: 25%
    - Skills: 20%
    - Certifications: 10%
    - Qualifications: 5%
    - Years of Experience: 5%
    - Domain Expertise: 3%
    - Position Name: 2%
    """
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=1,
        system="You are a helpful assistant that evaluates resumes based on job descriptions.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    response_text = response.content[0].text.strip()
    result = extract_score_and_explanation(response_text)
    return result

# Main function to process the data and generate scores and explanations
def main():
    input_file = "data.xlsx"
    output_file = "scores.xlsx"
    data = load_data(input_file)

    # Specify the row number to process (0-based index)
    row_number = int(input("Enter the row number to process: "))

    if 0 <= row_number < len(data):
        row = data.iloc[row_number]
        print(row)
        job_description = row["Job Description"]
        qualification = row["Qualifications"]
        work_experience = row["Previous Work Experience"]
        years_of_experience = row["Years of Experience"]
        domain_expertise = row["Domain Expertise"]
        position_name = row["Position Name"]
        projects_done = row["Projects Done"]
        skills = row["Skills"]
        certifications = row["Certifications"]
        result = generate_score_and_explanation(job_description, qualification, work_experience, years_of_experience, domain_expertise, position_name, projects_done, skills, certifications)
        data.at[row_number, "Score"] = result["score"]
        data.at[row_number, "Explanation"] = result["explanation"]
        save_scores(output_file, data)
        print(f"Score and explanation generated for row {row_number} and saved to {output_file}.")
    else:
        print("Invalid row number.")

if __name__ == "__main__":
    main()

Enter the row number to process: 52
S.No                                                                       27
Job Category                                                              MLE
Job Description             Job Title: Computer Vision Engineer\n\nRoles:\...
Resume                      Position Name: Computer Vision Engineer\n\nYea...
Qualifications              - M.S. in Computer Science from a reputable un...
Previous Work Experience    - 3 years as a Computer Vision Researcher at a...
Years of Experience                                                  5+ years
Domain Expertise            - Extensive experience in developing and deplo...
Position Name                                        Computer Vision Engineer
Projects Done               - Developed a state-of-the-art object detectio...
Skills                      - Proficient in Python, C++, and deep learning...
Certifications              - Deep Learning Specialization (Coursera)\n- A...
Relevance                   

# Dataset Curation Done -> Moving to converting the dataset to a Instruction Tuned format for the purpose of finetuning the language models

In [ ]:
import pandas as pd
import json

def convert_dataset(input_file, output_file):
    data = pd.read_excel(input_file)
    converted_data = []

    for _, row in data.iterrows():
        job_description = row["Job Description"]
        resume = row["Resume"]
        qualification = row["Qualifications"]
        work_experience = row["Previous Work Experience"]
        years_of_experience = row["Years of Experience"]
        domain_expertise = row["Domain Expertise"]
        position_name = row["Position Name"]
        projects_done = row["Projects Done"]
        skills = row["Skills"]
        certifications = row["Certifications"]
        score = row["Score"]
        explanation = row["Explanation"]

        instruction = f"Given this Job Description:\n{job_description}\n\nAnd Resume:\n{resume}\n\nExtract the following features from the resume (leave blank if not present):\n- Qualification\n- Previous Work Experience\n- Years of Experience\n- Domain Expertise\n- Position Name\n- Projects Done\n- Skills\n- Certifications\n\nAfter extracting the features, carefully compare each feature with the job description to assess the candidate's suitability for the role. The suitablity can be assessed in the following measure. \n\nGuidelines for scoring:\n- Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.\n- This idea is like conditional probability where, given the job description, the goal is to find the score of the resume.\n- Assess the relevance of the candidate's background to the specific needs mentioned in the job description.\n- Use the following weighted schema to prioritize different aspects of the resume:\n - Previous Work Experience: 30%\n - Projects Done: 25%\n - Skills: 20%\n - Certifications: 10%\n - Qualifications: 5%\n - Years of Experience: 5%\n - Domain Expertise: 3%\n - Position Name: 2%\n- Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.\n- Generate a detailed explanation highlighting the key factors on how each feature individually contributed to the overall score using the weighted schema approach.\n\nPlease provide the extracted features, explanation of how the weighted schema was applied to arrive at the score, and finally the score. Use the following format:\n{{\n \"extracted_features\": {{\n \"Qualification\": \"<qualification>\",\n \"Previous Work Experience\": \"<work_experience>\",\n \"Years of Experience\": \"<years_of_experience>\",\n \"Domain Expertise\": \"<domain_expertise>\",\n \"Position Name\": \"<position_name>\",\n \"Projects Done\": \"<projects_done>\",\n \"Skills\": \"<skills>\",\n \"Certifications\": \"<certifications>\"\n }},\n \"explanation\": \"<explanation>\",\n \"score\": <score>\n}}"

        response = f"{{\n \"The extracted features from the resume are as follows,\": {{\n \"Qualification\": \"{qualification}\",\n \"Previous Work Experience\": \"{work_experience}\",\n \"Years of Experience\": \"{years_of_experience}\",\n \"Domain Expertise\": \"{domain_expertise}\",\n \"Position Name\": \"{position_name}\",\n \"Projects Done\": \"{projects_done}\",\n \"Skills\": \"{skills}\",\n \"Certifications\": \"{certifications}\"\n }},\n \"explanation\": \"Using the weighted schema, the score calculated for individual features contributing to the total score can be explained as follows: {explanation}\\n\\nThe weighted scores from each category were summed up to arrive at the final score, which indicates the candidate's suitability for the given job description based on their resume.\",\n \"score\": {score}\n}}"


#       "inputs": f"<s>### Instruction:\n{instruction}\n\n### Response:\n{response}</s>"

        converted_row = {
            "text": f"‹user>: {Instruction} ‹bot>: {response}"
        }
        converted_data.append(converted_row)

        with open(output_file, "w") as file:
          json.dump(converted_data, file, indent=2)

    print(f"Dataset converted and saved to {output_file}.")
    print(f"Total number of rows in the converted dataset: {len(converted_data)}")

# Usage
input_file = "data.xlsx"
output_file = "converted_dataset.json"
convert_dataset(input_file, output_file)

Dataset converted and saved to converted_dataset.json.
Total number of rows in the converted dataset: 102


# SPLITTING THE JSON DATA INTO TRAIN AND TEST DATA

In [ ]:
import json

# Specify the path to your JSON file
json_file = "my.json"

# Specify the split ratio
train_ratio = 0.8  # 80% for training
test_ratio = 0.2   # 20% for testing

# Read the JSON file
with open(json_file, "r") as file:
    data = json.load(file)

# Calculate the number of samples for training and testing
total_samples = len(data)
train_samples = int(total_samples * train_ratio)
test_samples = total_samples - train_samples

# Split the data into training and test sets
train_data = data[:train_samples]
test_data = data[train_samples:]

# Save the training data to a new JSON file
with open("train.json", "w") as file:
    json.dump(train_data, file, indent=2)

# Save the test data to a new JSON file
with open("test.json", "w") as file:
    json.dump(test_data, file, indent=2)

print(f"Total samples: {total_samples}")
print(f"Training samples: {train_samples}")
print(f"Test samples: {test_samples}")
print("Training data saved to train.json")
print("Test data saved to test.json")

Total samples: 102
Training samples: 81
Test samples: 21
Training data saved to train.json
Test data saved to test.json


# CONVERTING XLSX TO JSON FORMAT

In [ ]:

import pandas as pd
import json

def convert_dataset(input_file, output_file):
    data = pd.read_excel(input_file)
    converted_data = []

    for _, row in data.iterrows():
        job_description = row["Job Description"]
        resume = row["Resume"]
        qualification = row["Qualifications"]
        work_experience = row["Previous Work Experience"]
        years_of_experience = row["Years of Experience"]
        domain_expertise = row["Domain Expertise"]
        position_name = row["Position Name"]
        projects_done = row["Projects Done"]
        skills = row["Skills"]
        certifications = row["Certifications"]
        score = row["Score"]
        explanation = row["Explanation"]

        instruction = f"Given this Job Description:\n{job_description}\n\nAnd Resume:\n{resume}\n\nExtract the following features from the resume (leave blank if not present):\n- Qualification\n- Previous Work Experience\n- Years of Experience\n- Domain Expertise\n- Position Name\n- Projects Done\n- Skills\n- Certifications\n\nAfter extracting the features, carefully compare each feature with the job description to assess the candidate's suitability for the role. The suitablity can be assessed in the following measure. \n\nGuidelines for scoring:\n- Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.\n- This idea is like conditional probability where, given the job description, the goal is to find the score of the resume.\n- Assess the relevance of the candidate's background to the specific needs mentioned in the job description.\n- Use the following weighted schema to prioritize different aspects of the resume:\n - Previous Work Experience: 30%\n - Projects Done: 25%\n - Skills: 20%\n - Certifications: 10%\n - Qualifications: 5%\n - Years of Experience: 5%\n - Domain Expertise: 3%\n - Position Name: 2%\n- Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.\n- Generate a detailed explanation highlighting the key factors on how each feature individually contributed to the overall score using the weighted schema approach.\n\nPlease provide the extracted features, explanation of how the weighted schema was applied to arrive at the score, and finally the score. Use the following format:\n{{\n \"extracted_features\": {{\n \"Qualification\": \"<qualification>\",\n \"Previous Work Experience\": \"<work_experience>\",\n \"Years of Experience\": \"<years_of_experience>\",\n \"Domain Expertise\": \"<domain_expertise>\",\n \"Position Name\": \"<position_name>\",\n \"Projects Done\": \"<projects_done>\",\n \"Skills\": \"<skills>\",\n \"Certifications\": \"<certifications>\"\n }},\n \"explanation\": \"<explanation>\",\n \"score\": <score>\n}}"

        response = f"{{\n \"The extracted features from the resume are as follows,\": {{\n \"Qualification\": \"{qualification}\",\n \"Previous Work Experience\": \"{work_experience}\",\n \"Years of Experience\": \"{years_of_experience}\",\n \"Domain Expertise\": \"{domain_expertise}\",\n \"Position Name\": \"{position_name}\",\n \"Projects Done\": \"{projects_done}\",\n \"Skills\": \"{skills}\",\n \"Certifications\": \"{certifications}\"\n }},\n \"explanation\": \"Using the weighted schema, the score calculated for individual features contributing to the total score can be explained as follows: {explanation}\\n\\nThe weighted scores from each category were summed up to arrive at the final score, which indicates the candidate's suitability for the given job description based on their resume.\",\n \"score\": {score}\n}}"


        converted_row = {
            "input_ids": f"<s>### Instruction:\n{instruction}\n\n### Response:\n{response}</s>"
        }
        converted_data.append(converted_row)

        with open(output_file, "w") as file:
          json.dump(converted_data, file, indent=2)

    print(f"Dataset converted and saved to {output_file}.")
    print(f"Total number of rows in the converted dataset: {len(converted_data)}")

# Usage
input_file = "data.xlsx"
output_file = "converted_dataset.json"
convert_dataset(input_file, output_file)

Dataset converted and saved to converted_dataset.json.
Total number of rows in the converted dataset: 102


In [ ]:
import pandas as pd
import json

def convert_dataset(input_file, output_file):
    data = pd.read_excel(input_file)
    converted_data = []

    for _, row in data.iterrows():
        job_description = row["Job Description"]
        resume = row["Resume"]
        qualification = row["Qualifications"]
        work_experience = row["Previous Work Experience"]
        years_of_experience = row["Years of Experience"]
        domain_expertise = row["Domain Expertise"]
        position_name = row["Position Name"]
        projects_done = row["Projects Done"]
        skills = row["Skills"]
        certifications = row["Certifications"]
        score = row["Score"]
        explanation = row["Explanation"]

        instruction = f"Given this Job Description: {job_description}\n\nAnd Resume:\n{resume}\n\nExtract the following features from the resume (leave blank if not present):\n- Qualification\n- Previous Work Experience\n- Years of Experience\n- Domain Expertise\n- Position Name\n- Projects Done\n- Skills\n- Certifications\n\nAfter extracting the features, carefully compare each feature with the job description to assess the candidate's suitability for the role. The suitablity can be assessed in the following measure. \n\nGuidelines for scoring:\n- Consider the alignment of the candidate's skills, experience, qualifications, and projects with the job requirements.\n- This idea is like conditional probability where, given the job description, the goal is to find the score of the resume.\n- Assess the relevance of the candidate's background to the specific needs mentioned in the job description.\n- Use the following weighted schema to prioritize different aspects of the resume:\n - Previous Work Experience: 30%\n - Projects Done: 25%\n - Skills: 20%\n - Certifications: 10%\n - Qualifications: 5%\n - Years of Experience: 5%\n - Domain Expertise: 3%\n - Position Name: 2%\n- Provide a score between 0 and 1, where 0 indicates no relevance and 1 indicates a perfect match.\n- Generate a detailed explanation highlighting the key factors on how each feature individually contributed to the overall score using the weighted schema approach.\n\nPlease provide the extracted features, explanation of how the weighted schema was applied to arrive at the score, and finally the score. Use the following format:\n{{\n \"extracted_features\": {{\n \"Qualification\": \"<qualification>\",\n \"Previous Work Experience\": \"<work_experience>\",\n \"Years of Experience\": \"<years_of_experience>\",\n \"Domain Expertise\": \"<domain_expertise>\",\n \"Position Name\": \"<position_name>\",\n \"Projects Done\": \"<projects_done>\",\n \"Skills\": \"<skills>\",\n \"Certifications\": \"<certifications>\"\n }},\n \"explanation\": \"<explanation>\",\n \"score\": <score>\n}}"

        response = f"{{\n \"The extracted features from the resume are as follows,\": {{\n \"Qualification\": \"{qualification}\",\n \"Previous Work Experience\": \"{work_experience}\",\n \"Years of Experience\": \"{years_of_experience}\",\n \"Domain Expertise\": \"{domain_expertise}\",\n \"Position Name\": \"{position_name}\",\n \"Projects Done\": \"{projects_done}\",\n \"Skills\": \"{skills}\",\n \"Certifications\": \"{certifications}\"\n }},\n \"explanation\": \ The breakdown of the weighted schema contributing to the overall calculated score can be explained as follows: {explanation}\\n\\n The weighted scores from each category were summed up to arrive at the final score, which indicates the candidate's suitability for the given job description based on their resume.\",\n \"score\": {score}\n}}"

        converted_row = {"input_text": f"{instruction}", "output_text": f"{response}"}
        converted_data.append(converted_row)



    with open(output_file, "w") as file:
        json.dump(converted_data, file, indent=2)

    print(f"Dataset converted and saved to {output_file}.")
    print(f"Total number of rows in the converted dataset: {len(converted_data)}")

# Usage
input_file = "data.xlsx"
output_file = "converted_dataset.json"
convert_dataset(input_file, output_file)


Dataset converted and saved to converted_dataset.json.
Total number of rows in the converted dataset: 102


In [ ]:
import pandas as pd
# assumes your JSON data is stored in a file named 'data.json'
data_json = pd.read_json('LLAMA.json')
print(type(data_json['inputs']))

#data_json['inputs'].to_json('json_data.jsonl')

<class 'pandas.core.series.Series'>


# Visualization of the Final Dataset Curated in Instruction Format

In [ ]:
data =  data_json['inputs'].to_frame()
print(data)

                                                inputs
0    <s>[INST] <<SYS>>\nYour an assitant that extra...
1    <s>[INST] <<SYS>>\nYour an assitant that extra...
2    <s>[INST] <<SYS>>\nYour an assitant that extra...
3    <s>[INST] <<SYS>>\nYour an assitant that extra...
4    <s>[INST] <<SYS>>\nYour an assitant that extra...
..                                                 ...
97   <s>[INST] <<SYS>>\nYour an assitant that extra...
98   <s>[INST] <<SYS>>\nYour an assitant that extra...
99   <s>[INST] <<SYS>>\nYour an assitant that extra...
100  <s>[INST] <<SYS>>\nYour an assitant that extra...
101  <s>[INST] <<SYS>>\nYour an assitant that extra...

[102 rows x 1 columns]


In [ ]:
json_data = pd.DataFrame(data)
print(json_data)

                                                inputs
0    <s>[INST] <<SYS>>\nYour an assitant that extra...
1    <s>[INST] <<SYS>>\nYour an assitant that extra...
2    <s>[INST] <<SYS>>\nYour an assitant that extra...
3    <s>[INST] <<SYS>>\nYour an assitant that extra...
4    <s>[INST] <<SYS>>\nYour an assitant that extra...
..                                                 ...
97   <s>[INST] <<SYS>>\nYour an assitant that extra...
98   <s>[INST] <<SYS>>\nYour an assitant that extra...
99   <s>[INST] <<SYS>>\nYour an assitant that extra...
100  <s>[INST] <<SYS>>\nYour an assitant that extra...
101  <s>[INST] <<SYS>>\nYour an assitant that extra...

[102 rows x 1 columns]


In [ ]:
json_l = json_data.to_json(orient='records', lines = True)
print(json_l)

print(type(json_l))

#jsonl_data = json_l.to_json('data.jsonl')

{"inputs":"<s>[INST] <<SYS>>\nYour an assitant that extract relevant features from the resume and give an explanation and score based on a weighted schema for the resume given a job description \n<<\/SYS>>\n\n Given this Job Description: What you will be doing:  Develop, deploy and optimize models to enable AI capabilities and improve user experience Collaborate with a team of Machine learning, software and product experts to bring your model from idea to the product through micro-service-based systems Work in an Enterprise SaaS platform in an Agile, cloud-based environment, and follow and ensure best engineering practices Utilize and be exposed to a wide range of cutting edge technologies such as: Front end: React, Typescripts, Graph QL, JavaScript Back End: C#\/.NET, Python, Kubernetes, Docker, Apache Kafka Cloud: Azure, AWS, Google Cloud, IBM DevOps: Terraform, Kubernetes, Ansible, Jenkins, Python AI\/ML: LLMs (Azure, Google, etc.), Vertext AI, Azure AI, IBM watsonx, huggingface, op

In [ ]:
import json

# Read data from JSON file
with open('data.json', 'r') as file:
    data = json.load(file)

# Function to convert data to JSON Lines format
def convert_to_jsonl(data):
    json_lines = []
    for item in data:
        json_lines.append(json.dumps(item))
    return '\n'.join(json_lines)

# Convert data to JSON Lines format
jsonl_data = convert_to_jsonl(data)

# Write JSON Lines data to a file
with open('converted_dataset.jsonl', 'w') as file:
    file.write(jsonl_data)


In [ ]:
# Read and print contents of JSON Lines file
with open('GCP.jsonl', 'r') as file:
    for line in file:
        print(line)

{"input_text": "Given this Job Description: What you will be doing:  Develop, deploy and optimize models to enable AI capabilities and improve user experience Collaborate with a team of Machine learning, software and product experts to bring your model from idea to the product through micro-service-based systems Work in an Enterprise SaaS platform in an Agile, cloud-based environment, and follow and ensure best engineering practices Utilize and be exposed to a wide range of cutting edge technologies such as: Front end: React, Typescripts, Graph QL, JavaScript Back End: C#/.NET, Python, Kubernetes, Docker, Apache Kafka Cloud: Azure, AWS, Google Cloud, IBM DevOps: Terraform, Kubernetes, Ansible, Jenkins, Python AI/ML: LLMs (Azure, Google, etc.), Vertext AI, Azure AI, IBM watsonx, huggingface, open source models   What you bring to the team:  Graduating from a STEM program in Machine Learning, AI, Data Science, or Computer Science in Spring 2025 Excellent interpersonal skills, strong part

In [ ]:
# Count the number of lines in a file
def count_lines(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for _ in file)

# Provide the file path
file_path = 'GCP.jsonl'

# Get the total number of lines
total_lines = count_lines(file_path)

print("Total number of lines:", total_lines)


Total number of lines: 102


# CONVERTING JSONL TO TRAIN AND TEST DATA

In [ ]:
# CONVERTIN
import json
import random

# Function to split data into training and testing sets
def split_data(file_path, train_ratio):
    # Read data from JSON Lines file
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]

    # Shuffle the data
    random.shuffle(data)

    # Calculate split index
    split_index = int(len(data) * train_ratio)

    # Split the data into training and testing sets
    train_data = data[:split_index]
    test_data = data[split_index:]

    return train_data, test_data

# Define the file path
file_path = 'converted_dataset.jsonl'

# Define the ratio for training data
train_ratio = 0.9

# Split the data
train_data, test_data = split_data(file_path, train_ratio)

# Write training data to a file
with open('train_data.jsonl', 'w') as file:
    for item in train_data:
        json.dump(item, file)
        file.write('\n')

# Write testing data to a file
with open('test_data.jsonl', 'w') as file:
    for item in test_data:
        json.dump(item, file)
        file.write('\n')

In [ ]:
import json
import csv

def jsonl_to_csv(jsonl_file, csv_file):
    # Open the JSONL file for reading
    with open(jsonl_file, 'r') as json_file:
        # Open the CSV file for writing
        with open(csv_file, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)

            # Write the header based on the keys of the first JSON object
            first_line = json_file.readline()
            if first_line:
                first_obj = json.loads(first_line)
                header = list(first_obj.keys())
                writer.writerow(header)

                # Rewind the file pointer
                json_file.seek(0)

                # Write each JSON object as a row in the CSV file
                for line in json_file:
                    obj = json.loads(line)
                    writer.writerow([str(obj[key]) for key in header])

# Example usage:
jsonl_file = 'test_data.jsonl'
csv_file = 'data.csv'
jsonl_to_csv(jsonl_file, csv_file)
